In [1]:
import os
import librosa
import numpy as np

import matplotlib.pyplot as plt
import IPython.display as ipd
import csv
from tqdm.auto import tqdm

import pandas as pd
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
from tensorflow.keras import backend as K

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from tensorflow.keras import models, layers
import tensorflow as tf
import random as python_random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

c:\Users\Usuario\anaconda3\envs\aine\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Descargar dataset y preprocesar

In [2]:
# from datasets import load_dataset

# dataset = load_dataset("ccmusic-database/music_genre", name="default")
# print(len(dataset["train"]))
# print(len(dataset["validation"]))
# print(len(dataset["test"]))

In [3]:
# dataset

In [4]:
"""
import librosa
import numpy as np
import csv

header = 'filename chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

# Función para procesar y escribir los datos de audio
def process_and_write_data(data, writer, label):
    audio_data = data['audio']['array']
    sr = data['audio']['sampling_rate']
    chroma_stft = librosa.feature.chroma_stft(y=audio_data, sr=sr)
    rms = librosa.feature.rms(y=audio_data)
    spec_cent = librosa.feature.spectral_centroid(y=audio_data, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=audio_data, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y=audio_data)
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sr)
    to_append = f'{label} {np.mean(chroma_stft)} {np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    to_append += f' {label}'
    writer.writerow(to_append.split())

# Procesar cada partición del dataset
for partition in tqdm(['train', 'validation', 'test']):

    with open(f'data_{partition}.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)

        for data in tqdm(dataset[partition]):
            process_and_write_data(data, writer, data['fst_level_label'])
"""

"\nimport librosa\nimport numpy as np\nimport csv\n\nheader = 'filename chroma_stft rms spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'\nfor i in range(1, 21):\n    header += f' mfcc{i}'\nheader += ' label'\nheader = header.split()\n\n# Función para procesar y escribir los datos de audio\ndef process_and_write_data(data, writer, label):\n    audio_data = data['audio']['array']\n    sr = data['audio']['sampling_rate']\n    chroma_stft = librosa.feature.chroma_stft(y=audio_data, sr=sr)\n    rms = librosa.feature.rms(y=audio_data)\n    spec_cent = librosa.feature.spectral_centroid(y=audio_data, sr=sr)\n    spec_bw = librosa.feature.spectral_bandwidth(y=audio_data, sr=sr)\n    rolloff = librosa.feature.spectral_rolloff(y=audio_data, sr=sr)\n    zcr = librosa.feature.zero_crossing_rate(y=audio_data)\n    mfcc = librosa.feature.mfcc(y=audio_data, sr=sr)\n    to_append = f'{label} {np.mean(chroma_stft)} {np.mean(rms)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)}

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

def prepare_dataset(filepath, scaler=None, fit_scaler=False):
    # Leer dataset asegurándonos de que no se tomen filas iniciales como cabecera si no son parte de los datos
    data = pd.read_csv(filepath, header=0)  # Asume que la primera fila es la cabecera
    
    # Verificar si alguna fila tiene 'chroma_stft' o cualquier valor no numérico en las columnas esperadas a ser numéricas
    # Esto es útil para capturar errores en los datos
    for col in data.columns[:-1]:  # Excluye la última columna que es de etiquetas
        if data[col].dtype == 'object':
            # Intenta convertir columnas a numérico, forzando errores a NaN para luego eliminar esas filas
            data[col] = pd.to_numeric(data[col], errors='coerce')
    
    # Eliminar filas con valores NaN resultantes de la conversión fallida de arriba
    data.dropna(inplace=True)
    
    # Preprocesar dataset: eliminar columna 'filename' si existe
    if 'filename' in data.columns:
        data = data.drop(['filename'], axis=1)
    
    # Preparar las etiquetas
    genre_list = data.iloc[:, -1]
    encoder = LabelEncoder()
    y = encoder.fit_transform(genre_list)
    
    # Preparar los datos de entrada (features)
    if scaler is None:
        scaler = MinMaxScaler()
    X_data = np.array(data.iloc[:, :-1], dtype=float)
    if fit_scaler:
        X = scaler.fit_transform(X_data)
    else:
        X = scaler.transform(X_data)
    
    return X, y, encoder, scaler

# Asegúrate de tener las rutas correctas a tus archivos CSV aquí
X_train, y_train, encoder_train, scaler = prepare_dataset('data_train.csv', fit_scaler=True)
X_val, y_val, encoder_validation, _ = prepare_dataset('data_validation.csv', scaler=scaler, fit_scaler=False)
X_test, y_test, encoder_test, _ = prepare_dataset('data_test.csv', scaler=scaler, fit_scaler=False)


In [6]:
# Asegurándose de que los datos ya han sido preparados con el código anterior

# Función para contar las ocurrencias de cada etiqueta
def count_labels(y, encoder):
    # Convertir las etiquetas numéricas a sus categorías originales
    labels = encoder.inverse_transform(y)
    unique, counts = np.unique(labels, return_counts=True)
    label_counts = dict(zip(unique, counts))
    return label_counts

# Contar etiquetas en el conjunto de entrenamiento
train_label_counts = count_labels(y_train, encoder_train)
print("Distribución de etiquetas en el conjunto de entrenamiento:")
print(train_label_counts)

# Contar etiquetas en el conjunto de validación
val_label_counts = count_labels(y_val, encoder_validation)
print("\nDistribución de etiquetas en el conjunto de validación:")
print(val_label_counts)

# Contar etiquetas en el conjunto de prueba
test_label_counts = count_labels(y_test, encoder_test)
print("\nDistribución de etiquetas en el conjunto de prueba:")
print(test_label_counts)

Distribución de etiquetas en el conjunto de entrenamiento:
{0: 329, 1: 1041}

Distribución de etiquetas en el conjunto de validación:
{0: 36, 1: 135}

Distribución de etiquetas en el conjunto de prueba:
{0: 40, 1: 132}


## Construir modelo

In [7]:

f1_scores = {}

X_trainval = np.concatenate((X_train, X_val))
y_trainval = np.concatenate((y_train, y_val))

# Mapping model names to their scikit-learn classes
model_map = {
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier()
}

param_grid = {
    'SVM': {'kernel': ['rbf', 'linear'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto']},
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]},
    'KNN': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']},
    'Gradient Boosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1]}
}

best_models = {}

for name, model in model_map.items():
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid[name], cv=5, scoring='f1')
    
    grid_search.fit(X_trainval, y_trainval)
    
    print(f'{name} best parameters: {grid_search.best_params_}')
    
    best_models[name] = grid_search.best_estimator_
        
    f1 = grid_search.best_score_ 
    f1_scores[name] = f1
    print(f'{name} f1 with best parameters: {f1:.4f}')



SVM best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
SVM f1 with best parameters: 0.9958
Logistic Regression best parameters: {'C': 10}
Logistic Regression f1 with best parameters: 0.9915
Random Forest best parameters: {'max_depth': 20, 'n_estimators': 50}
Random Forest f1 with best parameters: 0.9932
KNN best parameters: {'n_neighbors': 3, 'weights': 'uniform'}
KNN f1 with best parameters: 0.9923
Gradient Boosting best parameters: {'learning_rate': 0.1, 'n_estimators': 100}
Gradient Boosting f1 with best parameters: 0.9932


In [8]:
best_model_name = max(f1_scores, key=f1_scores.get)
best_model = best_models[best_model_name]  # Acceder al modelo usando el nombre

y_pred = best_model.predict(X_test)
y_pred_rounded = np.round(y_pred).astype(int)
f1 = f1_score(y_test, y_pred_rounded)

print(f'\nThe overall best performing model is {best_model_name} with an F1 score of {f1:.4f}.')



The overall best performing model is SVM with an F1 score of 1.0000.


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from focal_loss import BinaryFocalLoss


num_labels = len(np.unique(y_train))  # Esto debería ser 2 para clasificación binaria

# 'balanced' automáticamente ajusta los pesos inversamente proporcionales a las frecuencias de clase
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)

# class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}

# print("Pesos de Clase:", class_weights_dict)

model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels if num_labels > 2 else 1, activation='sigmoid' if num_labels == 2 else 'softmax')
])

# Ajustar la métrica F1 para clasificación binaria
def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)  # Redondear las predicciones para calcular la métrica
    # Asegurar que y_true y y_pred tengan el mismo tipo
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    tp = K.sum(y_true*y_pred, axis=0)
    fp = K.sum((1-y_true)*y_pred, axis=0)
    fn = K.sum(y_true*(1-y_pred), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# Elegir la función de pérdida según el problema
loss_function = 'binary_crossentropy' if num_labels == 2 else 'sparse_categorical_crossentropy'

# Compilar el modelo
model.compile(optimizer='adam',
              loss=BinaryFocalLoss(gamma=2),
              metrics=['accuracy',f1_score])

# Entrenamiento del modelo
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=700,
                    callbacks=[callback],
                    batch_size=X_train.shape[0],  # Ajustar el tamaño del lote si es necesario
                    # class_weight=class_weights_dict,
                    )

# Evaluar el modelo
model.evaluate(X_test, y_test)

c:\Users\Usuario\anaconda3\envs\aine\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/700
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.2978 - f1_score: 0.1491 - loss: 0.4347 - val_accuracy: 0.2105 - val_f1_score: 0.0000e+00 - val_loss: 0.2338
Epoch 2/700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.3088 - f1_score: 0.2070 - loss: 0.3607 - val_accuracy: 0.2105 - val_f1_score: 0.0000e+00 - val_loss: 0.2004
Epoch 3/700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.3445 - f1_score: 0.2782 - loss: 0.3048 - val_accuracy: 0.2164 - val_f1_score: 0.2809 - val_loss: 0.1797
Epoch 4/700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 0.3752 - f1_score: 0.3494 - loss: 0.2629 - val_accuracy: 0.7193 - val_f1_score: 0.7608 - val_loss: 0.1675
Epoch 5/700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.4387 - f1_score: 0.4376 - loss: 0.2334 - val_accuracy: 0.7895 - val_f1_score: 0.7895 - val_loss: 0.1595
Epoch 6/700
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.4679 - f1_score: 0.4859 - loss: 0.2195 - val_accuracy: 0.7895 - val_f1_score: 0.7895 -

[0.15062452852725983, 0.7674418687820435, 0.7569444179534912]